## Diyabet
Bu veri seti aslen Ulusal Diyabet ve Sindirim ve Böbrek Hastalıkları Enstitüsü'nden alınmıştır. Amaç, bir hastanın diyabetli olup olmadığını tanısal ölçümlere dayanarak tahmin etmektir.

## İçerik
Bu örneklerin daha büyük bir veritabanından seçilmesine çeşitli kısıtlamalar getirildi. Özellikle, buradaki tüm hastalar, Pima Kızılderili mirasına sahip en az 21 yaşında kadınlardır.

- Pregnancies: Hamile kalma sayısı
- Glucose:  Plazma glikoz konsantrasyonu, bir oral glikoz tolerans testinde 2 saat
- BloodPressure: Diyastolik kan basıncı (mm Hg)
- SkinThickness: Triceps deri kıvrım kalınlığı (mm)
- Insulin: 2 saatlik serum insülini (mu U/ml)
- BMI: Vücut kitle indeksi (kg cinsinden ağırlık/(m cinsinden boy)^2)
- DiabetesPedigreeFunction: Diyabet soyağacı işlevi
- Age: Yaş (yıl)
- Outcome: Sınıf değişkeni (0 veya 1)

İlgili bilgiler:
  Bu örneklerin seçimine çeşitli kısıtlamalar getirildi. 
  Özellikle, buradaki tüm hastalar kadındır.
  
- Örnek Sayısı: 768
- Nitelik Sayısı: 8 artı sınıf

Her Nitelik için: (tümü sayısal değerli)
- Hamile kalma sayısı
- Plazma glikoz konsantrasyonu, bir oral glikoz tolerans testinde 2 saat
- Diyastolik kan basıncı (mm Hg)
- Triceps deri kıvrım kalınlığı (mm)
- 2 saatlik serum insülini (mu U/ml)
- Vücut kitle indeksi (kg cinsinden ağırlık/(m cinsinden boy)^2)
- Diyabet soyağacı işlevi Yaşam yılları)
- Sınıf değişkeni (0 veya 1)

## Missing Values (Eksik Özellik Değerleri): Evet
## Sınıf Dağılımı: (sınıf değeri 1, " için şeker hastalığı pozitif test edildi" olarak yorumlanır")

## 1. Installing

In [ ]:
# Bu Python 3 ortamı, yüklü birçok yardımcı analiz kitaplığı ile birlikte gelir
# Kaggle/python Docker görüntüsü ile tanımlanır: https://github.com/kaggle/docker-python
# Örneğin, yüklenecek birkaç yardımcı paket var

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Giriş verileri dosyaları salt okunur "../input/" dizininde bulunur
# Örneğin, bunu çalıştırmak (çalıştır'ı tıklatarak veya Shift+Enter tuşlarına basarak) giriş dizini altındaki tüm dosyaları listeleyecektir.

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# "Save & Run All" seçeneğini kullanarak bir sürüm oluşturduğunuzda çıktı olarak korunan geçerli dizine (/kaggle/working/) 5GB'a kadar yazabilirsiniz.
# /kaggle/temp/ dizinine geçici dosyalar da yazabilirsiniz, ancak bunlar geçerli oturumun dışında kaydedilmez

import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error, r2_score, roc_auc_score, roc_curve, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 

%config InlineBackend.figure_format = 'retina'

# tüm sütunları ve satırları görüntülemek için:
pd.set_option('display.max_columns', None); pd.set_option('display.max_rows', None);  # to display all columns and rows
pd.set_option('display.float_format', lambda x: '%.2f' % x) # The number of numbers that will be shown after the comma.


## 2. EDA (Exploratory of Data Analysis) (Veri Analizinin Keşfi)

### 2.1. Data Preperation (Veri Hazırlama)

In [ ]:
#Veri setini okuma
df = pd.read_csv("../input/diabetes-data-set/diabetes.csv")

In [ ]:
#Veri Setinin boyutu
df.shape

In [ ]:
#belirtilen değerlere göre veri setiyle ilgili istatistiki veriler
df.describe([0.10,0.25,0.50,0.75,0.90,0.95,0.99]).T

In [ ]:
##Veri setinin yüzde kaçı diabet hastası(1) yüzde kaçı diabet hastası değil(0)
df["Outcome"].value_counts()*100/len(df)

In [ ]:
#Diabet olan ve diabet olmayanların sayısını öğrendik. 268 kişi diabet hastası ve 500 kişi diabet hastası değil
df.Outcome.value_counts()

In [ ]:
#Yaş aralığına göre dağılım
df["Age"].hist(edgecolor = "black");

## 3. Missing Value Analysis ((Eksik Değer Analizi)

In [ ]:
#Burda her kolonda kaç boş değer var bunun kontrolünü yaptık. Hiç boş değer görünmüyor.
df.isnull().sum()

In [ ]:
#Burda ise verdiğimiz sütunlarda değeri 0 olan alanları NaN olarak değiştirdik.
df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0, np.NaN)

In [ ]:
#Her kolondaki boş değerlerin toplamı
df.isnull().sum()

In [ ]:
#Her sütun için dolu ve eksik değerleri gösteren dağılım grafiği
import missingno as msno
msno.bar(df);

In [ ]:
def carp(x,y):
    
    z = x*y
    
    return z


In [ ]:
carp(4,5)

In [ ]:
# Eksik değerler bu fonksiyon ile her bir değişkenin medyan değerleri ile doldurulacaktır.

def median_target(var):   
    
    temp = df[df[var].notnull()]
    
    temp = temp[[var, 'Outcome']].groupby(['Outcome'])[[var]].median().reset_index()
    
    return temp

In [ ]:
columns = df.columns

columns = columns.drop("Outcome")

In [ ]:
columns

In [ ]:
# median_target fonksiyonu ile eksik değerlere dolu değerlerin medyanını atadık ve outcome'a göre grupladık.
median_target('Glucose')

In [ ]:
# Eksik gözlemler için verilecek değerlere hasta olmayanların ortanca değeri ve hasta olanların ortanca değerleri verilir.
columns = df.columns

columns = columns.drop("Outcome")

for col in columns:
    
    df.loc[(df['Outcome'] == 0 ) & (df[col].isnull()), col] = median_target(col)[col][0]
    df.loc[(df['Outcome'] == 1 ) & (df[col].isnull()), col] = median_target(col)[col][1]

In [ ]:
#Hasta olmayanların(0) Pregnancies(Gebelik) değeri boş olanları gösteriyor. Yokmuş.
df.loc[(df['Outcome'] == 0 ) & (df["Pregnancies"].isnull()), "Pregnancies"]

In [ ]:
#Hasta olmayanların(0) BloodPressure(diyastolik kan basıncı) değeri boş olanları gösteriyor.
df[(df['Outcome'] == 0 ) & (df["BloodPressure"].isnull())]

## 4. Outliers Analysis

In [ ]:
Q1 = df["BloodPressure"].quantile(0.25) #İstenen sütunda verilen nicelikteki değerleri döndürür yani BloodPressure alanı 0.25 olanları döndürecek.
Q3 = df["BloodPressure"].quantile(0.75) #Burdada BloodPressure alanı 0.75 olanları döndürecek.
IQR = Q3-Q1 #0.75 değerlerin sayısı - 0.25 değerler
lower = Q1 - 1.5*IQR #0.25 değerler - 1.5*aradaki fark(IQR)
upper = Q3 + 1.5*IQR #0.75 değerler - 1.5*aradaki fark(IQR)

In [ ]:
lower

In [ ]:
upper

In [ ]:
#Burda .any ile 104.0 değerinden daha büyük değerler var mı yok mu onu kontrol ettik ve True döndürdü yani daha büyük değer var. 
df[(df["BloodPressure"] > upper)].any(axis=None) 

In [ ]:
for feature in df:
    print(feature)

In [ ]:
for feature in df:
    
    Q1 = df[feature].quantile(0.05)
    Q3 = df[feature].quantile(0.95)
    IQR = Q3 - Q1
    lower = Q1 - 1.5*IQR
    upper = Q3 + 1.5*IQR
    
    if df[(df[feature] > upper)].any(axis=None):
        print(feature,"yes")
    else:
        print(feature, "no")

In [ ]:
df.head()

In [ ]:
df.shape

## 5. Feature Engineering

In [ ]:
# BMI'ye göre bazı aralıklar belirlendi ve kategorik değişkenler atandı.
NewBMI = pd.Series(["Underweight", "Normal", "Overweight", "Obesity 1", "Obesity 2", "Obesity 3"], dtype = "category")
#Yukarda kategoriler belirlendi ve aşağıdaki kodlarla hangi aralığa hangi kategori geleceği belirlendi.
df["NewBMI"] = NewBMI

df.loc[df["BMI"] < 18.5, "NewBMI"] = NewBMI[0]

df.loc[(df["BMI"] > 18.5) & (df["BMI"] <= 24.9), "NewBMI"] = NewBMI[1]
df.loc[(df["BMI"] > 24.9) & (df["BMI"] <= 29.9), "NewBMI"] = NewBMI[2]
df.loc[(df["BMI"] > 29.9) & (df["BMI"] <= 34.9), "NewBMI"] = NewBMI[3]
df.loc[(df["BMI"] > 34.9) & (df["BMI"] <= 39.9), "NewBMI"] = NewBMI[4]
df.loc[df["BMI"] > 39.9 ,"NewBMI"] = NewBMI[5]

In [ ]:
df.head()

In [ ]:
def set_insulin(row):
    if row["Insulin"] >= 16 and row["Insulin"] <= 166: #df insulin değeri 16 dan büyükeşit ve 166 dan küçükeşitse Normal yazdır değilse Abnormal yazdır.
        return "Normal"
    else:
        return "Abnormal"     

In [ ]:
df.head()

In [ ]:
#NewInsulinScore adında yeni bir sütunda insülini normal ve abnormal olanların değerlerini gösterdik.
df["NewInsulinScore"] = df.apply(set_insulin, axis=1) 

In [ ]:
df.head()

In [ ]:
#df.drop("NewInsulinScore", inplace = True, axis = 1)
#df.head()

In [ ]:
# Glikoz değişkenine göre bazı aralıklar belirlenmiş ve bunlara kategorik değişkenler atanmıştır.
NewGlucose = pd.Series(["Low", "Normal", "Overweight", "Secret", "High"], dtype = "category")

df["NewGlucose"] = NewGlucose

df.loc[df["Glucose"] <= 70, "NewGlucose"] = NewGlucose[0]

df.loc[(df["Glucose"] > 70) & (df["Glucose"] <= 99), "NewGlucose"] = NewGlucose[1]

df.loc[(df["Glucose"] > 99) & (df["Glucose"] <= 126), "NewGlucose"] = NewGlucose[2]

df.loc[df["Glucose"] > 126 ,"NewGlucose"] = NewGlucose[3]

In [ ]:
df.head()

## 6. One-hot Encoding

In [ ]:
df = pd.get_dummies(df, columns =["NewBMI","NewInsulinScore", "NewGlucose"], drop_first = True)

In [ ]:
df.head()

In [ ]:
categorical_df = df[['NewBMI_Obesity 1','NewBMI_Obesity 2', 'NewBMI_Obesity 3', 'NewBMI_Overweight','NewBMI_Underweight',
                     'NewInsulinScore_Normal','NewGlucose_Low','NewGlucose_Normal', 'NewGlucose_Overweight', 'NewGlucose_Secret']]

## 7. Feature Standartization

In [ ]:
y = df["Outcome"]
X = df.drop(["Outcome",'NewBMI_Obesity 1','NewBMI_Obesity 2', 'NewBMI_Obesity 3', 'NewBMI_Overweight','NewBMI_Underweight',
                     'NewInsulinScore_Normal','NewGlucose_Low','NewGlucose_Normal', 'NewGlucose_Overweight', 'NewGlucose_Secret'], axis = 1)
cols = X.columns
index = X.index

In [ ]:
y.head()

In [ ]:
X.head()

In [ ]:
cols

In [ ]:
index

In [ ]:
from sklearn.preprocessing import RobustScaler
transformer = RobustScaler().fit(X)
X = transformer.transform(X)
X = pd.DataFrame(X, columns = cols, index = index)

In [ ]:
X.head()

In [ ]:
X = pd.concat([X, categorical_df], axis = 1)

In [ ]:
X.head()

## 8. Model

In [ ]:
models = []
models.append(('LR', LogisticRegression(random_state = 12345)))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier(random_state = 12345)))
models.append(('RF', RandomForestClassifier(random_state = 12345)))
models.append(('SVM', SVC(gamma='auto', random_state = 12345)))
models.append(('XGB', GradientBoostingClassifier(random_state = 12345)))
models.append(("LightGBM", LGBMClassifier(random_state = 12345)))

# sırayla her modeli değerlendirin
results = []
names = []

for name, model in models:
    
        kfold = KFold(n_splits = 10, random_state = 12345)
        
        cv_results = cross_val_score(model, X, y, cv = 10, scoring= "accuracy")
        results.append(cv_results)
        names.append(name)
        msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        print(msg)
        
# boxplot algorithm comparison
fig = plt.figure(figsize=(15,10))
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()



## 9. Model Optimizasyonu (Model Tunning)

### 9.1. Random Forests Tuning

In [ ]:
rf_params = {"n_estimators" :[100,200,500,1000], 
             "max_features": [3,5,7], 
             "min_samples_split": [2,5,10,30],
            "max_depth": [3,5,8,None]}



In [ ]:
rf_model = RandomForestClassifier(random_state = 12345)

In [ ]:
gs_cv = GridSearchCV(rf_model, 
                    rf_params,
                    cv = 10,
                    n_jobs = -1,
                    verbose = 2).fit(X, y)


In [ ]:
gs_cv.best_params_

### 9.1.1. RF Final Model

In [ ]:
rf_tuned = RandomForestClassifier(**gs_cv.best_params_)

In [ ]:
rf_tuned = rf_tuned.fit(X,y)

In [ ]:
cross_val_score(rf_tuned, X, y, cv = 10).mean()

In [ ]:
feature_imp = pd.Series(rf_tuned.feature_importances_,
                        index=X.columns).sort_values(ascending=False)

sns.barplot(x=feature_imp, y=feature_imp.index)
plt.xlabel('Significance Score Of Variables')
plt.ylabel('Variables')
plt.title("Variable Severity Levels")
plt.show()

### 9.2. LightGBM Model Tuning

In [ ]:
lgbm = LGBMClassifier(random_state = 12345)

In [ ]:
lgbm_params = {"learning_rate": [0.01, 0.03, 0.05, 0.1, 0.5],
              "n_estimators": [500, 1000, 1500],
              "max_depth":[3,5,8]}


In [ ]:
gs_cv = GridSearchCV(lgbm, 
                     lgbm_params, 
                     cv = 10, 
                     n_jobs = -1, 
                     verbose = 2).fit(X, y)

In [ ]:
gs_cv.best_params_

### 9.2.1 LightGBM Final Model

In [ ]:
lgbm_tuned = LGBMClassifier(**gs_cv.best_params_).fit(X,y)

In [ ]:
cross_val_score(lgbm_tuned, X, y, cv = 10).mean()

In [ ]:
feature_imp = pd.Series(lgbm_tuned.feature_importances_,
                        index=X.columns).sort_values(ascending=False)

sns.barplot(x=feature_imp, y=feature_imp.index)
plt.xlabel('Significance Score Of Variables')
plt.ylabel('Variables')
plt.title("Variable Severity Levels")
plt.show()

### 9.3. XGBoost Model Tuning

In [ ]:
xgb = GradientBoostingClassifier(random_state = 12345)

In [ ]:
xgb_params = {
    "learning_rate": [0.01, 0.1, 0.2, 1],
    "min_samples_split": np.linspace(0.1, 0.5, 10),
    "max_depth":[3,5,8],
    "subsample":[0.5, 0.9, 1.0],
    "n_estimators": [100,1000]}

In [ ]:
xgb_cv_model  = GridSearchCV(xgb,xgb_params, cv = 10, n_jobs = -1, verbose = 2).fit(X, y)

In [ ]:
xgb_cv_model.best_params_

### 9.3.1. XGBoost Final Model

In [ ]:
xgb_tuned = GradientBoostingClassifier(**xgb_cv_model.best_params_).fit(X,y)

In [ ]:
cross_val_score(xgb_tuned, X, y, cv = 10).mean()

In [ ]:
feature_imp = pd.Series(xgb_tuned.feature_importances_,
                        index=X.columns).sort_values(ascending=False)

sns.barplot(x=feature_imp, y=feature_imp.index)
plt.xlabel('Significance Score Of Variables')
plt.ylabel('Variables')
plt.title("Variable Severity Levels")
plt.show()

## 10. Comparison of Final Models

In [ ]:
models = []

models.append(('RF', RandomForestClassifier(random_state = 12345, max_depth = 8, max_features = 7, min_samples_split = 2, n_estimators = 500)))
models.append(('XGB', GradientBoostingClassifier(random_state = 12345, learning_rate = 0.1, max_depth = 5, min_samples_split = 0.1, n_estimators = 100, subsample = 1.0)))
models.append(("LightGBM", LGBMClassifier(random_state = 12345, learning_rate = 0.01,  max_depth = 3, n_estimators = 1000)))

# sırayla her modeli değerlendirin
results = []
names = []

In [ ]:
for name, model in models:
    
        kfold = KFold(n_splits = 10, random_state = 12345)
        cv_results = cross_val_score(model, X, y, cv = 10, scoring= "accuracy")
        results.append(cv_results)
        names.append(name)
        msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        print(msg)
        
# kutu grafiği algoritması karşılaştırması
fig = plt.figure(figsize=(15,10))
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

## 11. Sonuç

Bu çalışmanın amacı, diyabet veri seti için sınıflandırma modelleri oluşturmak ve modeller kurarak bir kişinin hasta olup olmadığını tahmin etmek ve kurulan modellerde maksimum validasyon puanlarını elde etmektir. Yapılan iş aşağıdaki gibidir:

1) Diyabet Veri Seti okundu.

2) Keşfedici Veri Analizi ile; 
Veri setinin yapısal verileri kontrol edildi. Veri setindeki değişken türleri incelenmiştir. Veri setinin boyut bilgilerine ulaşıldı. Veri setindeki 0 değerleri eksik değerlerdir. Öncelikle bu 0 değerleri NaN değerleri ile değiştirildi. Veri setinin tanımlayıcı istatistikleri incelendi.

3) Veri Ön İşleme bölümü; 
df for: NaN değerleri eksik gözlemler, her bir değişkenin hasta olup olmadığına ilişkin medyan değerlerle dolduruldu. Aykırı değerler LOF tarafından belirlendi ve düşürüldü. X değişkenleri rubost yöntemi ile standardize edilmiştir.

4) Model Oluşturma Sırasında; 
Lojistik Regresyon, KNN, SVM, CART, Random Forests, XGBoost, LightGBM gibi makine öğrenme modelleri kullanılarak Çapraz Doğrulama Puanı hesaplanmıştır. Daha sonra Rastgele Ormanlar, XGBoost, LightGBM hiperparametre optimizasyonları Çapraz Doğrulama değerini artırmak için optimize edildi.

5) Sonuç; 
LightBM hiperparametre optimizasyonu sonucunda oluşturulan model, Çapraz Doğrulama Puanı değeri en yüksek model olmuştur. (0.89)

